# Import Widget Libraries

In [1]:
%load_ext autoreload   
%autoreload 2
import magneton
from magneton.core.idom_loader import reload_bundle
from magneton.core.widget import WidgetModel

magneton_client: 0.0.0


# Pre-process Data

In [2]:
import pandas as pd

In [3]:
def transform_data(data_url):
    df = pd.read_csv(data_url)
    df = df[df['Year'] == 2017]
    df = df[['Cohort','State','Drivers']]
    df = df.pivot_table(index=['State'], 
            columns=['Cohort'], values='Drivers').fillna(0)
    df = df.reset_index()
    return df

In [4]:
data_url ='https://data.transportation.gov/api/views/xfkb-3bxx/rows.csv?accessType=DOWNLOAD'

state_data_df = transform_data(data_url)

# Define Actions

In [5]:
def get_data_table(df, start_letter=None):
    ls = list(df['State'].unique())
    if start_letter:
        ls = [x for x in ls if x.startswith(start_letter)]
    return ls

In [6]:
def get_distribution_by_state(df, state_name=None):
    result = []
    if state_name:
        row = df[df['State'] == state_name]
        dist = row.loc[:, row.columns != 'State'].to_dict('records')[0]
        for key, value in dist.items():
            result.append({"x": key, "y": value})
    else:
        ls = df.columns
        _df = df.mean(axis=0, numeric_only=True)
        for key in ls:
            if key != 'State':
                result.append({"x": key, "y": _df[key]})
    return result

In [7]:
def get_filtered_distribution_by_buckets(df, state_name=None):
    dist = get_distribution_by_state(df, state_name)
    filtered_ls = []
    for _d in dist:
        if "-" in _d['x']:
            filtered_ls.append(_d)
    return filtered_ls

# Customize All Components and Actions

In [8]:
from magneton import FullyCustomizableLinkedViews
    
def init(): 
    # Fetch initial view data
    return {
        "distribution": get_filtered_distribution_by_buckets(state_data_df),
        "index": -1,
        "table": get_data_table(state_data_df, 'C')
    }

def select(element):
    # implement selection
    dist = get_filtered_distribution_by_buckets(state_data_df, element)
    index = get_data_table(state_data_df, 'C').index(element)
    return {
            "distribution": dist,
            "index": index
            }
# Create widget 
lv_widget = FullyCustomizableLinkedViews(state_data_df, 'State', {"init": init, "select": select})

In [9]:
lv_widget.show()

component(10b499b80, self=<magneton.core.widget.StatefulWidgetBase.StatefulWidgetBase object at 0x1033c3100>)

LayoutWidget(Layout(component(10b499b80, self=<magneton.core.widget.StatefulWidgetBase.StatefulWidgetBase obje…